In [159]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
from webdriver_manager.chrome import ChromeDriverManager
import requests
import pandas as pd
import re
import selenium
from pandas import DataFrame

In [2]:
def init_browser():
    # @NOTE: Replace the path with your actual path to the chromedriver
    executable_path = {"executable_path": ChromeDriverManager().install()}
    return Browser("chrome", **executable_path, headless=False)

In [102]:
browser = init_browser()
url = "https://www.akc.org/dog-breeds/"
browser.visit(url)

#THERE ARE 24 PAGES OF DOG BREEDS
#https://www.akc.org/dog-breeds/page/24/

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [/Users/jenniferdean/.wdm/drivers/chromedriver/mac64/87.0.4280.88/chromedriver] found in cache


In [153]:
# breeds_list_of_dicts = []

for p in range(9,11):
    color_list = []
    final_color_list = []
    p += 1
    #####GO TO NEW BREED PAGE#####
    url = f"https://www.akc.org/dog-breeds/page/{p}/"
    browser.visit(url)
    
    for i in range(12):
        html = browser.html
        soup = bs(html, "html.parser")
        
        #####GO TO BREED DETAIL PAGE#####
        browser.find_by_css('a[class="d-block relative"]')[i].click()
        time.sleep(1)
        
        html = browser.html
        soup = bs(html, "html.parser")
        
        
        #####COLLECT DOG BREED NAME#####
        dog_breed = soup.find("h1").get_text()
        dog_breed = dog_breed.replace('\n', '').strip(" ")
        
        #####COLLECT DOG ATTRIBUTES#####
        attribute_block = soup.find_all("span", class_="attribute-list__description")
        
        if len(attribute_block) == 6:
            temperment = soup.find("span", class_="attribute-list__description attribute-list__text attribute-list__text--lg mb4 bpm-mb5 pb0 d-block").get_text()
            akc_rank = soup.find_all("span", class_="attribute-list__description attribute-list__text")[0].get_text()
            height = soup.find_all("span", class_="attribute-list__description attribute-list__text")[1].get_text()
            weight = soup.find_all("span", class_="attribute-list__description attribute-list__text")[2].get_text()
            life_ex = soup.find_all("span", class_="attribute-list__description attribute-list__text")[3].get_text()
            group = soup.find_all("span", class_="attribute-list__description attribute-list__text")[4].get_text()
        elif len(attribute_block) == 5: 
            temperment = soup.find("span", class_="attribute-list__description attribute-list__text attribute-list__text--lg mb4 bpm-mb5 pb0 d-block").get_text()
            height = soup.find_all("span", class_="attribute-list__description attribute-list__text")[0].get_text()
            weight = soup.find_all("span", class_="attribute-list__description attribute-list__text")[1].get_text()
            life_ex = soup.find_all("span", class_="attribute-list__description attribute-list__text")[2].get_text()
            group = soup.find_all("span", class_="attribute-list__description attribute-list__text")[3].get_text()
            akc_rank = "NA"
        elif len(attribute_block) == 4:
            temperment = soup.find("span", class_="attribute-list__description attribute-list__text attribute-list__text--lg mb4 bpm-mb5 pb0 d-block").get_text()
            height = soup.find_all("span", class_="attribute-list__description attribute-list__text")[0].get_text()
            life_ex = soup.find_all("span", class_="attribute-list__description attribute-list__text")[1].get_text()
            group = soup.find_all("span", class_="attribute-list__description attribute-list__text")[2].get_text()
            akc_rank = "NA"
            weight = "NA"
        else:
            print(f"Attributes are missing for '{dog_breed}'")
            temperment = f"Attributes are missing for '{dog_breed}'"
            akc_rank = f"Attributes are missing for '{dog_breed}'"
            height = f"Attributes are missing for '{dog_breed}'"
            weight = f"Attributes are missing for '{dog_breed}'"
            life_ex = f"Attributes are missing for '{dog_breed}'"
            group = f"Attributes are missing for '{dog_breed}'"
            
        #####COLLECT DOG IMAGE#####    
        try:
            image_url = soup.find_all('img', class_="media-wrap__image lozad")[1]["data-src"]
        except:
            print(f"Image is missing for '{dog_breed}'")
            image_url = f"Image is missing for '{dog_breed}'"
            
        #####COLLECT DOG DESCRIPTION#####
        try:
            description = soup.find("div", class_="breed-hero__footer").get_text()
            description = description.replace('\n', '').strip(" ")
        except:
            print(f"Description is missing for '{dog_breed}'")
            description = f"Description is missing for '{dog_breed}'"
            
        #####COLLECT DOG CARE SCALES#####    
        try:
            brushing = soup.find_all("div", class_="bar-graph__section")[0]["style"].strip('width: ;')
            shedding = soup.find_all("div", class_="bar-graph__section")[1]["style"].strip('width: ;')
            energy = soup.find_all("div", class_="bar-graph__section")[2]["style"].strip('width: ;')
            trainability = soup.find_all("div", class_="bar-graph__section")[3]["style"].strip('width: ;')
            temperment_scale = soup.find_all("div", class_="bar-graph__section")[4]["style"].strip('width: ;')
        except IndexError:
            print(f"Missing 'Care' Scales for '{dog_breed}'")
            brushing = f"Missing 'Care' Scales for '{dog_breed}'"
            shedding = f"Missing 'Care' Scales for '{dog_breed}'"
            energy = f"Missing 'Care' Scales for '{dog_breed}'"
            trainability = f"Missing 'Care' Scales for '{dog_breed}'"
            temperment_scale = f"Missing 'Care' Scales for '{dog_breed}'"
        
        #####COLLECT DOG COLORS #####
        try:
            table = soup.find_all('table')[0]
        
            color_list = []
            final_color_list = []

            for row in table.find_all('td'):
                if 'Description' in row:
                    continue
                elif 'Standard Colors' in row:
                    continue
                elif 'Registration Code' in row:
                    continue
                elif "Check Mark For Standard Color" in row:
                    continue
                elif "\n" in row:
                    continue
                else:
                    color = row.get_text()
                    color_list.append(color)
            for i in color_list:
                if "Check Mark For Standard Color" in i:
                    continue
                if "\n" in i:
                    continue
                else:
                    try:
                        if int(i) > 0:
                            continue
                    except:
                        final_color_list.append(i)
        except:
            print(f"No Colors for '{dog_breed}'")
        
        #####GO BACK TO BREED PAGE#####
        browser.back()
        
        #####APPEND TO THE DICTIONARY#####
        breeds_list_of_dicts.append({"breed_name" : dog_breed,
                         "temperment": temperment,
                         "image" : image_url,
                         "description" : description,
                         "akc_rank" : akc_rank,
                         "height" : height,
                         "weight" : weight,
                         "life_expectancy" : life_ex,
                         "brushing_scale" : brushing,
                         "shedding_scale" : shedding,
                         "energy_scale" : energy,
                         "trainability_scale" : trainability,
                         "temperment_scale" : temperment_scale,
                         "color_options" : final_color_list
                        })
        time.sleep(1)
        
browser.quit()

Missing 'Care' Scales for 'French Spaniel'
Missing 'Care' Scales for 'German Longhaired Pointer'
Missing 'Care' Scales for 'Giant Schnauzer'


ElementClickInterceptedException: Message: element click intercepted: Element <a href="https://www.akc.org/dog-breeds/havanese/" class="d-block relative">...</a> is not clickable at point (479, 824). Other element would receive the click: <div class="breed-compare-filter cmw py3" style="transition: none;transform: translateY(-41px);">...</div>
  (Session info: chrome=87.0.4280.88)


In [ ]:
#Missing 'Care' Scales for 'Irish Red and White Setter'
#Missing 'Care' Scales for 'Irish Terrier'
#Missing 'Care' Scales for 'Jagdterrier'
#Missing 'Care' Scales for 'Hokkaido'
#No Colors for 'Japanese Terrier'
#Missing 'Care' Scales for 'Norrbottenspets'
#Missing 'Care' Scales for 'Perro de Presa Canario'
#Missing 'Care' Scales for 'Poodle (Miniature)'
#Missing 'Care' Scales for 'Porcelaine'
#Missing 'Care' Scales for 'Portuguese Sheepdog'
#Missing 'Care' Scales for 'Pudelpointer'
#Missing 'Care' Scales for 'Pyrenean Mastiff'
#Missing 'Care' Scales for 'Rafeiro do Alentejo'
#Missing 'Care' Scales for 'Segugio Italiano'
#Missing 'Care' Scales for 'Shikoku'
#Missing 'Care' Scales for 'Slovensky Cuvac'
#Missing 'Care' Scales for 'Slovensky Kopov'
#Missing 'Care' Scales for 'Spanish Mastiff'
#Missing 'Care' Scales for 'Spinone Italiano'
#Missing 'Care' Scales for 'Tornjak'
#Missing 'Care' Scales for 'Transylvanian Hound'
#Missing 'Care' Scales for 'Treeing Tennessee Brindle'
#Missing 'Care' Scales for 'Welsh Springer Spaniel'
#Missing 'Care' Scales for 'American Hairless Terrier'
#Missing 'Care' Scales for 'Bedlington Terrier'
#Missing 'Care' Scales for 'Braque du Bourbonnais'
#Missing 'Care' Scales for 'Braque Francais Pyrenean'
#Missing 'Care' Scales for 'Broholmer'
#Missing 'Care' Scales for 'Catahoula Leopard Dog'
#Missing 'Care' Scales for 'Danish-Swedish Farmdog'
#Missing 'Care' Scales for 'Deutscher Wachtelhund'
#Missing 'Care' Scales for 'Dogo Argentino'
#Missing 'Care' Scales for 'Estrela Mountain Dog'
#Missing 'Care' Scales for 'French Spaniel'
#Missing 'Care' Scales for 'German Longhaired Pointer'
#Missing 'Care' Scales for 'Giant Schnauzer'
#Missing 'Care' Scales for 'Hokkaido'

In [154]:
breeds_list_of_dicts

[{'breed_name': 'Hovawart',
  'temperment': 'Alert, Faithful, Intelligent',
  'image': 'https://s3.amazonaws.com/cdn-origin-etr.akc.org/wp-content/uploads/2017/11/02174018/Hovawart.10.jpg',
  'description': 'The Hovawart is a medium-temperament working dog with versatile usage and has a very good nose. He is kind, has an even disposition and protective instinct, is self-confident and has the ability to take stress. His balanced body proportions and special devotion to his family make him an outstanding companion, watch, guard, tracking and rescue-dog.',
  'akc_rank': 'NA',
  'height': '23-28 inches',
  'weight': '65-90 pounds',
  'life_expectancy': '10-14 years',
  'brushing_scale': '20%',
  'shedding_scale': '60%',
  'energy_scale': '60%',
  'trainability_scale': '20%',
  'temperment_scale': '40%',
  'color_options': ['Black', 'Black', 'Black', 'Blonde', 'Black', 'Blonde']},
 {'breed_name': 'Ibizan Hound',
  'temperment': 'Family-Oriented, Even-Tempered, Polite',
  'image': 'https://s

In [157]:
len(breeds_list_of_dicts) #Missing 'Care' Scales for 'Hokkaido'

269

In [88]:
html = browser.html
soup = bs(html, "html.parser")

In [156]:
html = browser.html
soup = bs(html, "html.parser")

#####COLLECT DOG BREED NAME#####
dog_breed = soup.find("h1").get_text()
dog_breed = dog_breed.replace('\n', '').strip(" ")

#####COLLECT DOG ATTRIBUTES#####
attribute_block = soup.find_all("span", class_="attribute-list__description")

if len(attribute_block) == 6:
    temperment = soup.find("span", class_="attribute-list__description attribute-list__text attribute-list__text--lg mb4 bpm-mb5 pb0 d-block").get_text()
    akc_rank = soup.find_all("span", class_="attribute-list__description attribute-list__text")[0].get_text()
    height = soup.find_all("span", class_="attribute-list__description attribute-list__text")[1].get_text()
    weight = soup.find_all("span", class_="attribute-list__description attribute-list__text")[2].get_text()
    life_ex = soup.find_all("span", class_="attribute-list__description attribute-list__text")[3].get_text()
    group = soup.find_all("span", class_="attribute-list__description attribute-list__text")[4].get_text()
elif len(attribute_block) == 5: 
    temperment = soup.find("span", class_="attribute-list__description attribute-list__text attribute-list__text--lg mb4 bpm-mb5 pb0 d-block").get_text()
    height = soup.find_all("span", class_="attribute-list__description attribute-list__text")[0].get_text()
    weight = soup.find_all("span", class_="attribute-list__description attribute-list__text")[1].get_text()
    life_ex = soup.find_all("span", class_="attribute-list__description attribute-list__text")[2].get_text()
    group = soup.find_all("span", class_="attribute-list__description attribute-list__text")[3].get_text()
    akc_rank = "NA"
elif len(attribute_block) == 4:
    temperment = soup.find("span", class_="attribute-list__description attribute-list__text attribute-list__text--lg mb4 bpm-mb5 pb0 d-block").get_text()
    height = soup.find_all("span", class_="attribute-list__description attribute-list__text")[0].get_text()
    life_ex = soup.find_all("span", class_="attribute-list__description attribute-list__text")[1].get_text()
    group = soup.find_all("span", class_="attribute-list__description attribute-list__text")[2].get_text()
    akc_rank = "NA"
    weight = "NA"
else:
    print(f"Attributes are missing for '{dog_breed}'")
    temperment = f"Attributes are missing for '{dog_breed}'"
    akc_rank = f"Attributes are missing for '{dog_breed}'"
    height = f"Attributes are missing for '{dog_breed}'"
    weight = f"Attributes are missing for '{dog_breed}'"
    life_ex = f"Attributes are missing for '{dog_breed}'"
    group = f"Attributes are missing for '{dog_breed}'"

#####COLLECT DOG IMAGE#####    
try:
    image_url = soup.find_all('img', class_="media-wrap__image lozad")[1]["data-src"]
except:
    print(f"Image is missing for '{dog_breed}'")
    image_url = f"Image is missing for '{dog_breed}'"

#####COLLECT DOG DESCRIPTION#####
try:
    description = soup.find("div", class_="breed-hero__footer").get_text()
    description = description.replace('\n', '').strip(" ")
except:
    print(f"Description is missing for '{dog_breed}'")
    description = f"Description is missing for '{dog_breed}'"

#####COLLECT DOG CARE SCALES#####    
try:
    brushing = soup.find_all("div", class_="bar-graph__section")[0]["style"].strip('width: ;')
    shedding = soup.find_all("div", class_="bar-graph__section")[1]["style"].strip('width: ;')
    energy = soup.find_all("div", class_="bar-graph__section")[2]["style"].strip('width: ;')
    trainability = soup.find_all("div", class_="bar-graph__section")[3]["style"].strip('width: ;')
    temperment_scale = soup.find_all("div", class_="bar-graph__section")[4]["style"].strip('width: ;')
except IndexError:
    print(f"Missing 'Care' Scales for '{dog_breed}'")
    brushing = f"Missing 'Care' Scales for '{dog_breed}'"
    shedding = f"Missing 'Care' Scales for '{dog_breed}'"
    energy = f"Missing 'Care' Scales for '{dog_breed}'"
    trainability = f"Missing 'Care' Scales for '{dog_breed}'"
    temperment_scale = f"Missing 'Care' Scales for '{dog_breed}'"

#####COLLECT DOG COLORS #####
try:
    table = soup.find_all('table')[0]

    color_list = []
    final_color_list = []

    for row in table.find_all('td'):
        if 'Description' in row:
            continue
        elif 'Standard Colors' in row:
            continue
        elif 'Registration Code' in row:
            continue
        elif "Check Mark For Standard Color" in row:
            continue
        elif "\n" in row:
            continue
        else:
            color = row.get_text()
            color_list.append(color)
        for i in color_list:
            if "Check Mark For Standard Color" in i:
                continue
            if "\n" in i:
                continue
            else:
                try:
                    if int(i) > 0:
                        continue
                except:
                    final_color_list.append(i)
except:
    print(f"No Colors for '{dog_breed}'")

#####GO BACK TO BREED PAGE#####
browser.back()

#####APPEND TO THE DICTIONARY#####
breeds_list_of_dicts.append({"breed_name" : dog_breed,
                 "temperment": temperment,
                 "image" : image_url,
                 "description" : description,
                 "akc_rank" : akc_rank,
                 "height" : height,
                 "weight" : weight,
                 "life_expectancy" : life_ex,
                 "brushing_scale" : brushing,
                 "shedding_scale" : shedding,
                 "energy_scale" : energy,
                 "trainability_scale" : trainability,
                 "temperment_scale" : temperment_scale,
                 "color_options" : final_color_list
                })
time.sleep(1)

#browser.quit()

Missing 'Care' Scales for 'Hokkaido'


In [164]:
breeds_df = DataFrame(breeds_list_of_dicts)

In [165]:
breeds_df

,breed_name,temperment,image,description,akc_rank,height,weight,life_expectancy,brushing_scale,shedding_scale,energy_scale,trainability_scale,temperment_scale,color_options
0,Hovawart,"Alert, Faithful, Intelligent",https://s3.amazonaws.com/cdn-origin-etr.akc.or...,The Hovawart is a medium-temperament working d...,NA,23-28 inches,65-90 pounds,10-14 years,20%,60%,60%,20%,40%,"[Black, Black, Black, Blonde, Black, Blonde]"
1,Ibizan Hound,"Family-Oriented, Even-Tempered, Polite",https://s3.amazonaws.com/cdn-origin-etr.akc.or...,The Ibizan Hound is a lithe and leggy visitor ...,Ranks 152 of 197,"23.5-27.5 inches (male), 22.5-26 inches (female)","50 pounds (male), 45 pounds (female)",11-14 years,20%,40%,100%,40%,40%,"[Red, Red, Red, Red & White, Red, Red & White,..."
2,Icelandic Sheepdog,"Friendly, Playful, Inquisitive",https://s3.amazonaws.com/cdn-origin-etr.akc.or...,"The Icelandic Sheepdog, Iceland’s only native ...",Ranks 155 of 197,"18 inches (male), 16.5 inches (female)","30 pounds (male), 25 pounds (female)",12-14 years,40%,60%,80%,100%,80%,"[Black & White, Black & White, Black & White, ..."
3,Irish Red and White Setter,"Courageous, Spirited, Determined",https://s3.amazonaws.com/cdn-origin-etr.akc.or...,The rollicking Irish Red and White Setter is a...,Ranks 146 of 197,"24.5-26 inches (male), 22.5-24 inches (female)","42-60 pounds (male), 35-50 pounds (female)",11-15 years,Missing 'Care' Scales for 'Irish Red and White...,Missing 'Care' Scales for 'Irish Red and White...,Missing 'Care' Scales for 'Irish Red and White...,Missing 'Care' Scales for 'Irish Red and White...,Missing 'Care' Scales for 'Irish Red and White...,"[White, White]"
4,Irish Setter,"Active, Outgoing, Sweet-Natured",https://s3.amazonaws.com/cdn-origin-etr.akc.or...,The Irish Setter is a high-spirited gundog kno...,Ranks 77 of 197,"27 inches (male), 25 inches (female)","70 pounds (male), 60 pounds (female)",12-15 years,60%,60%,100%,60%,100%,"[Chestnut, Chestnut, Chestnut, Mahogany, Chest..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264,Hamiltonstovare,"Agile, Versatile, Regal",https://s3.amazonaws.com/cdn-origin-etr.akc.or...,The Hamiltonstovare is a versatile scent hound...,NA,19-24 inches,40-75 pounds,14-17 years,20%,20%,80%,60%,80%,[Tri-Colored]
265,Hanoverian Scenthound,"Loyal, Independent, Calm",https://s3.amazonaws.com/cdn-origin-etr.akc.or...,The Hanoverian Scenthound has a calm and assur...,NA,19-21 inches,79-99 pounds,10-14 years,20%,20%,60%,40%,40%,"[Red, Brindle]"
266,Harrier,"Friendly, Outgoing, People-Oriented",https://s3.amazonaws.com/cdn-origin-etr.akc.or...,"The Harrier is a swift, prey-driven pack hound...",Ranks 189 of 197,19-21 inches,45-60 pounds,12-15 years,40%,40%,100%,80%,80%,"[Black White & Tan, Lemon & White, Red & White]"
267,Havanese,"Intelligent, Outgoing, Funny",https://s3.amazonaws.com/cdn-origin-etr.akc.or...,"Havanese, the only dog breed native to Cuba, a...",Ranks 24 of 197,8.5-11.5 inches,7-13 pounds,14-16 years,60%,40%,60%,80%,100%,"[Black, Black, Black, Black & Silver, Black, B..."


In [167]:
breeds_df.to_csv('breeds.csv', index=False)